In [2]:
import numpy as np
import pandas as pd
import matsim
import geojson
import yaml

# Path

In [8]:
root_path = r'../../../../data/intermediate/test/freight_emission/'
van_scenario_outputs = 'vanOutputs/'
cargo_bike_scenario_outputs = 'cargoBikeOutputs/'
output_path = '../../../../data/intermediate/test/freight_emission/simWrapperData/'

# Read events

In [25]:
emission_events = matsim.event_reader(root_path+cargo_bike_scenario_outputs+'output_events.xml.gz', types='coldEmissionEvent,warmEmissionEvent')
emission_events

<generator object event_reader at 0x000002AA47E00820>

# Convert to DataFrame

In [26]:
event_keys = set()
events_list = []
for emission_event in emission_events:
    # type(emission_event)
    # print(emission_event)
    events_list.append(emission_event)
    current_keys = list(emission_event.keys())
    event_keys.update(current_keys)
event_keys

{'BC_exhaust',
 'BC_non_exhaust',
 'Benzene',
 'CH4',
 'CO',
 'CO2_TOTAL',
 'CO2_rep',
 'CO2e',
 'FC',
 'FC_MJ',
 'HC',
 'N2O',
 'NH3',
 'NMHC',
 'NO2',
 'NOx',
 'PM',
 'PM2_5',
 'PM2_5_non_exhaust',
 'PM_non_exhaust',
 'PN',
 'Pb',
 'SO2',
 'linkId',
 'time',
 'type',
 'vehicleId'}

In [27]:
events_dict = {}
for idx, event in enumerate(events_list):
    event_dict = {}
    for key in event_keys:
        if key in event.keys():
            event_dict[key] = event[key]
        else:
            event_dict[key] = None
    events_dict[idx] = event_dict
    

In [28]:
events_df = pd.DataFrame.from_dict(events_dict, orient='index')
events_df

,NH3,PM2_5_non_exhaust,CH4,PN,PM_non_exhaust,SO2,CO,linkId,PM,BC_exhaust,...,PM2_5,NO2,type,FC_MJ,HC,BC_non_exhaust,Benzene,NMHC,CO2_rep,Pb
0,None,None,None,None,None,None,None,333784188_1,None,None,...,None,None,coldEmissionEvent,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,333784189_2,None,None,...,None,None,coldEmissionEvent,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,333784188_1,None,None,...,None,None,coldEmissionEvent,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,333784189_2,None,None,...,None,None,coldEmissionEvent,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,333784188_1,None,None,...,None,None,coldEmissionEvent,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15108,1.884581904052425E-5,8.752326586034533E-5,0.002837192190838657,5.300746122550251E9,1.6254320636283735E-4,9.200506600954581E-6,0.04728470777960894,326736776_2,0.0013047698640033852,2.355915135124238E-4,...,0.0013047698640033852,4.96882241946535E-5,warmEmissionEvent,0.02659573244511102,0.018834089430779778,1.0502791495679477E-5,0.0010461443694999904,0.01599689779888325,1.8234254819807314,0.0
15109,2.7241469569743602E-5,1.2651412912605635E-4,0.0038345217617899194,5.716085694007658E9,2.3495480882950327E-4,8.540235045716334E-6,0.05658721185118827,326736776_3,0.0018467236925033468,3.32497693835689E-4,...,0.0018467236925033468,6.487744658888157E-5,warmEmissionEvent,0.024505177214763956,0.024239255249004908,1.5181694905999445E-5,0.001303301759902464,0.0204047348337917,1.6925678089715808,0.0
15110,1.5558746113836312E-5,7.225752670376002E-5,0.0023423313615872285,4.376194155176785E9,1.3419254821932846E-4,7.59576147978047E-6,0.03903734625849463,326736776_4,0.0010771929310878431,1.9449982711986565E-4,...,0.0010771929310878431,4.102164323182883E-5,warmEmissionEvent,0.021956925721042606,0.015549062373393076,8.670902867975878E-6,8.636766918152967E-4,0.01320673147325773,1.5053850443237944,0.0
15111,3.390874314693319E-5,1.5747810880798092E-4,0.005104878755906618,9.537480879454443E9,2.924593419359866E-4,1.6554208362214738E-5,0.08507802220900192,333784188_0,0.0023476350955728476,4.2389307156733653E-4,...,0.0023476350955728476,8.940260054608206E-5,warmEmissionEvent,0.04785294066268081,0.03388763839562556,1.889737232364313E-5,0.0018822976408564068,0.02878276064540752,3.280837313735424,0.0


In [29]:
# Convert columns which is number-like str into float and ignore the rest
for column in events_df.columns:
    try:
        events_df[column] = events_df[column].astype(float)
    except:
        pass

# Analysis

## Aggregate by link

In [30]:
link_co2e = events_df.pivot_table(index='linkId',
                                       columns='type', 
                                       values='CO2e', 
                                       aggfunc='sum')
link_co2e

type,coldEmissionEvent,warmEmissionEvent
linkId,,
100344071_4,0.0,0.855291
100344071_r_2,NaN,0.863370
100344071_r_3,NaN,1.546372
100344071_r_4,NaN,0.966608
100344071_r_5,NaN,1.710581
...,...,...
splitLink_44932919_0_1,NaN,13.090518
splitLink_44932919_0_2,NaN,13.090518
splitLink_44932919_0_3,NaN,13.090518


In [31]:
like_co2e = link_co2e.fillna(0)
link_co2e['sum'] = link_co2e[['coldEmissionEvent', 'warmEmissionEvent']].sum(axis=1)
link_co2e 

type,coldEmissionEvent,warmEmissionEvent,sum
linkId,,,
100344071_4,0.0,0.855291,0.855291
100344071_r_2,NaN,0.863370,0.863370
100344071_r_3,NaN,1.546372,1.546372
100344071_r_4,NaN,0.966608,0.966608
100344071_r_5,NaN,1.710581,1.710581
...,...,...,...
splitLink_44932919_0_1,NaN,13.090518,13.090518
splitLink_44932919_0_2,NaN,13.090518,13.090518
splitLink_44932919_0_3,NaN,13.090518,13.090518


# Output 

In [32]:
link_co2e.to_csv(output_path+cargo_bike_scenario_outputs+'link_co2e.csv', index=True, sep=';', compression='gzip', encoding='utf-8-sig')

# Write yaml

In [96]:
data = {
    'header': {
        'tab': "Leuven Van Emissions Scenario",
        'title': "Overview of the Leuven Van Emissions Scenario",
        'description': "One carrier-one van with 200 shipments."
    },
    'layout': {
        'map': {
            None: {
            'type': '"links"',
            'title': "Overview",
            'description': "Simulated emissions of a van in Leuven, with 200 shipments.",
            'height': 12,
            'csvFile': "link_co2e.csv.gz",
            'geojsonFile': "GemeenteLeuvenWithHbefaType.geojson.gz",
            # 'center': [6.779091, 51.23347],
            'zoom': 11,
            'display': {
                'color': {
                    'dataset': "csvFile",
                    'columnName': "sum",
                    'colorRamp': {
                        'ramp': "Viridis",
                        'reverse': False,
                        'steps': 7
                    }
                },
                'width': {
                    'dataset': "csvFile",
                    'columnName': "sum",
                    'scaleFactor': 1000
                }
            }
        }
        }
    }
}

# # 自定义 Dumper，仅对 "links" 字符串加双引号
# class CustomDoubleQuotedDumper(yaml.SafeDumper):
#     def represent_str(self, data):
#         # 如果值是 "links"，则使用双引号输出
#         if data == "links":
#             return self.represent_scalar('tag:yaml.org,2002:str', data, style='"')
#         # 否则使用默认表示方式
#         return super().represent_str(data)
    
# 将数据写入 YAML 文件
with open(output_path+van_scenario_outputs+'testConfigV2.yaml', 'w', encoding='utf-8-sig') as file:
    yaml.dump(data, file, allow_unicode=True, default_flow_style=False)